In [1]:
#data stuff
import time
import pandas as pd
import numpy as np
import datetime as dt

#regression stuff
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

#graph stuff
import seaborn as sns
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import plotly
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'


#import functions from other file that we want
from utilities import *

# 0.1 Read in data

In [ ]:
#read in clickthru data from vw dataset
q = '''SELECT 
     account
    , attr_window
    , campaign
    , _match
    , campaign_objective
    , Year
    , Quarter
    , Month
    , week
    , week_start
    , date
    , marketing_initiative
    , marketing_segment
    , marketing_subinitiative
    , channel
    , platform
    , site
    , sum(case when sor_prod = 'All (Core All, NPV)' then spend else 0 end) spend
    , sum(case when sor_prod = 'All (Core All, NPV)' then clicks else 0 end) clicks
    , sum(case when sor_prod = 'All (Core All, NPV)' then impressions else 0 end) impressions
    , sum(case
            when sor_prod = 'Core' then conversions else null end) digi_ada_conversions
    , sum(case
            when sor_prod = 'Crosswords' then conversions else null end) games_conversions
    , sum(case
            when sor_prod = 'EDU' then conversions else null end) edu_conversions
    , sum(case
            when sor_prod = 'Home Delivery' then conversions else null end) hd_conversions
    , sum(case
            when sor_prod = 'CK' then conversions else null end) ck_conversions     
    , sum(case
            when sor_prod = 'Core All (Core, HD, EDU)' then conversions else null end) core_conversions
    , sum(case
            when sor_prod = 'All (Core All, NPV)' then conversions else null end) all_conversions
FROM `nyt-mkt-prd.paid_media_data.placement_daily_vw`
WHERE date >= '2020-01-01' and attr_window in('Media Reported - Click Thru')
group by 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13,14,15,16,17
order by date, campaign, _match
    '''
start_time = time.time() 

clickthru_df = pd.read_gbq(q,
                 project_id ='nyt-bigquery-beta-workspace',
                 dialect='standard')

print(f'time took: {str(round(time.time() - start_time, 2))}')

In [ ]:
clickthru_df.head()

In [ ]:
#read in viewthru data from vw dataset
q = '''SELECT 
     account
    , attr_window
    , campaign
    , _match
    , campaign_objective
    , Year
    , Quarter
    , Month
    , week
    , week_start
    , date
    , marketing_initiative
    , marketing_segment
    , marketing_subinitiative
    , channel
    , platform
    , site
    , sum(case when sor_prod = 'All (Core All, NPV)' then spend else 0 end) spend
    , sum(case when sor_prod = 'All (Core All, NPV)' then clicks else 0 end) clicks
    , sum(case when sor_prod = 'All (Core All, NPV)' then impressions else 0 end) impressions
    , sum(case
            when sor_prod = 'Core' then conversions else null end) digi_ada_conversions
    , sum(case
            when sor_prod = 'Crosswords' then conversions else null end) games_conversions
    , sum(case
            when sor_prod = 'EDU' then conversions else null end) edu_conversions
    , sum(case
            when sor_prod = 'Home Delivery' then conversions else null end) hd_conversions
    , sum(case
            when sor_prod = 'CK' then conversions else null end) ck_conversions     
    , sum(case
            when sor_prod = 'Core All (Core, HD, EDU)' then conversions else null end) core_conversions
    , sum(case
            when sor_prod = 'All (Core All, NPV)' then conversions else null end) all_conversions
FROM `nyt-mkt-prd.paid_media_data.placement_daily_vw`
WHERE date >= '2020-01-01' and attr_window in('Media Reported - View Thru')
group by 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13,14,15,16,17
order by date, campaign, _match
    '''
start_time = time.time() 

viewthru_df = pd.read_gbq(q,
                 project_id ='nyt-bigquery-beta-workspace',
                 dialect='standard')

print(f'time took: {str(round(time.time() - start_time, 2))}')

In [ ]:
#rename columns in the separate dataframes to identify clickthru and viewthru
#first clickthru
for col in clickthru_df.columns:
    if 'conversions' in col: 
        clickthru_df[col+'_ct'] = clickthru_df[col]
        clickthru_df.drop(col, axis=1, inplace=True)
        
#now viewthru
for col in viewthru_df.columns:
    if 'conversions' in col: 
        viewthru_df[col+'_vt'] = viewthru_df[col]
        viewthru_df.drop(col, axis=1, inplace=True)

In [ ]:
#now merge the dataframes 
#create a truncated version of viewthru DF to make the merge more efficient
vt_merge = viewthru_df[['account','campaign','_match','date','marketing_initiative','marketing_segment'] + [col for col in viewthru_df.columns if 'conversions' in col]]

#merge with clickthru df
df = clickthru_df.merge(vt_merge, how='left', on=[col for col in vt_merge.columns if 'conversions' not in col])

In [ ]:
df.shape

In [ ]:
# check to make sure merge worked properly
df[df.duplicated()]

# 0.2 Data Cleaning

## Data Removal

In [ ]:
#exclude Paid Search 
df = df.loc[df['channel'] != 'Paid Search']

In [ ]:
#exclude Brand, Branding, Retention, App Download, and App Install initatives bc not relevant to Chrome cookie loss
df = df[~df['marketing_initiative'].isin(['Brand','Branding','Retention','App Download', 'App-Install'])]

In [ ]:
#exclude where spend = 0 but impressions, clicks, or conversions are not equal to 0
bad_spend_df = df.loc[(df['spend'] == 0) & 
                    (
                        (df['impressions'] != 0) |
                        (df['clicks'] != 0) |
                        (df['all_conversions_vt'] != 0) | 
                        (df['all_conversions_ct'] != 0)
                        
                    )]
df = pd.concat([df,bad_spend_df]).drop_duplicates(keep=False)

## Impute X-UNKNOWN-X

In [ ]:
#replace all NaNs to 0 
df = df.fillna(0)

In [ ]:
#some Twigeo campaigns don't get recognized as Games campaigns in PMD, manually correct those
df.loc[(df['marketing_subinitiative'] == 'X-UNKNOWN-X') & 
       (
          (df['campaign'].str.contains('game')) | 
          (df['campaign'].str.contains('xwd')) | 
          (df['campaign'].str.contains('cross'))
       ), 'marketing_subinitiative'] = 'Games (former: Crosswords)'

## Low-Volume Grouping

In [ ]:
#group low-volume channels 
df['new_channel'] = df['channel']
df.loc[~df['channel'].isin(['Social','Display','Paid Search','Native']), 'new_channel'] = 'Low-Volume Channels'

In [ ]:
#group low-volume sites
df['new_site'] = df['site']
df.loc[~df['site'].isin(['The Trade Desk','DBM_US','Snap','Facebook','Twitter','Google AdWords (DSA)','Bing Ads', 'Google AdWords']), 'new_site'] = 'Low-Volume Sites'

In [ ]:
#group low-volume products
df['new_product'] = df['marketing_subinitiative']
df.loc[~df['marketing_subinitiative'].isin(['Home Delivery','Core','Games','Cooking']), 'new_product'] = 'Low-Volume Products'

In [ ]:
#check to see if the above groupings work
df['channel'].value_counts()
df['new_channel'].value_counts()
df['site'].value_counts()
df['new_site'].value_counts()
df['marketing_subinitiative'].value_counts()
df['new_product'].value_counts()

## Adding, Dropping, and Renaming Columns

In [ ]:
#group dates by bigger cols using the dt module 
#apply() -> so it reads better in graphs 

#create date-based aggregation columns for easier grouping
df['year_month_cal'] = df['date'].dt.to_period('M').apply(lambda r: r.start_time)
df['year_week_monday'] = df['date'].dt.to_period('W').apply(lambda r: r.start_time)
df['year_quarter'] = df['date'].dt.to_period('Q').apply(lambda r: r.start_time)

In [ ]:
#create a composite channel-marketing-initiative column for easier grouping by combos of channel and marketing_initiative 
df['channel_mkt_init'] = df['channel'] + " - " + df['marketing_initiative']

In [ ]:
#get rid of attr_window (bc it isn't needed anymore just says click thru)
df = df.drop(['attr_window'], axis=1)

In [ ]:
#add total conversions column
df['total_conversions'] = df['all_conversions_ct'] + df['all_conversions_vt']

# 0.3 Create Dataframes Split By Channel

In [ ]:
df['new_channel'].value_counts()

In [ ]:
df_low_volume = df[df['new_channel'] == 'Low-Volume Channels']
df_low_volume.shape

In [ ]:
df_display = df[df['channel'] == 'Display']
df_display.shape

In [ ]:
df_native = df[df['channel'] == 'Native']
df_native.shape

In [ ]:
df_display_native = df[(df['channel'] == 'Display') | (df['channel'] == 'Native')]
df_display_native.shape

In [ ]:
df_social = df[df['channel'] == 'Social']
df_social.shape

In [ ]:
df_social['site'].value_counts()

In [ ]:
df_fb = df_social[df_social['site'] == 'Facebook']
df_fb.shape

In [ ]:
df_snap = df_social[df_social['site'] == 'Snap']
df_snap.shape

In [ ]:
df_twitter= df_social[df_social['site'] == 'Twitter']
df_twitter.shape

# 1. Functions

## 1.1 Removal Functions

In [ ]:
#returns df with outliers, low-vol channels, sites, and products removed
# pretty much don't need -> move elsewhere 

#impact of removals: 

#if ALL (outliers, paid search, all 3 low vol) removed, then the df is cut in more than half (832638 -> 374316)
#Outliers (702,551)
#Paid Search (657,082)
#Total Conversions = 0 (613,001)
#Low-volume channels (772,840)
#low-volume sites (831,519)
#low-volume products (754,941)
#all 3 low-volume (733,921)

# some of it move to the top and remove once -> Paid Search
# channels where there is 0 spend -> remove it for each specific dataframe 

def removal(df):
    #remove_outliers
    df_spend = df
    q1, q3 = np.percentile(df_spend['spend'], [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    df_outliers_remov= df[df['spend'] > lower_bound][df['spend'] < upper_bound] 

    # df_spend['spend'].describe(percentile = )
    
    #remove Paid Search
    df_paid_search_remov = df_outliers_remov.loc[df['channel'] != 'Paid Search']
    
#     #remove low-volume channels
#     df_high_vol_channel = df_paid_search_remov.loc[df['new_channel'] != 'Low-Volume Channels']
    
#     #remove low-volume sites
#     df_high_vol_sites = df_high_vol_channel.loc[df['new_site'] != 'Low-Volume Sites']
    
#     #remove low-volume products
#     df_high_vol_products = df_high_vol_sites.loc[df['new_product'] != 'Low-Volume Products']
    
    #remove any June 2020 for Native -> infinity (spend = 0)
    df_remov_native_zero = df_high_vol_products.loc[~((df['year_month_cal'] == '2020-06-01') & 
                                             (df['channel'] == 'Native'))]
    
    #remove any June 2020 for Twitter -> infinity (spend = 0)
    df_remov_twitter_zero = df_remov_native_zero.loc[~((df['year_month_cal'] == '2020-06-01') & 
                                             (df['channel'] == 'Social') &
                                             (df['site'] == 'Twitter'))]
    #remove total conversions = 0 
    df_final_remov = df_remov_twitter_zero.loc[df['total_conversions'] == 0]
    
    return df_final_remov

## 1.2 Multiplier Functions

In [ ]:
#returns a df with the multiplier 
#more arguments -> training start, training end (date range to calculate multiplier)

def df_multiplier(df):
    df_copy = df
   # df_copy= dfc['sliced on the arg above']
    
    
    #drop cols you don't want in solution attempt -> maybe don't need to drop them 
#     df_copy = df_copy.drop(['account','campaign','_match','campaign_objective','Month','Quarter','week','week_start','date','marketing_initiative',
#               'marketing_segment','platform',
#              'digi_ada_conversions_ct','games_conversions_ct','edu_conversions_ct','hd_conversions_ct','ck_conversions_ct','core_conversions_ct',
#              'digi_ada_conversions_vt','games_conversions_vt','edu_conversions_vt','hd_conversions_vt','ck_conversions_vt','core_conversions_vt',
#              'year_week_monday','channel_mkt_init','new_channel','new_site','new_product'], axis=1)
#    df_copy.reset_index(inplace = True)
    
    #anywhere where year_month_cal -> substitute for time granularity variable (column to be grouped by -> date, week, month)
    
    #group by channel, month, year
    df_copy = df_copy.groupby(['year_month_cal']).sum()
    
    #create vt percentage column -> change name 
    df_copy['avg_monthly_vt_pctg'] = df_copy['all_conversions_vt'] / df_copy['total_conversions']
    
    #create ct percentage column
    df_copy['avg_monthly_ct_pctg'] = df_copy['all_conversions_ct'] / df_copy['total_conversions']
    
    #fix the nulls in vt pctg col
    df_copy.loc[(df_copy['total_conversions'] == 0), 'avg_monthly_vt_pctg'] = 0
    
    #fix the nulls in ct pctg col
    df_copy.loc[(df_copy['total_conversions'] == 0), 'avg_monthly_ct_pctg'] = 0
    
    #create overall ct column
    df_copy['overall_avg_ct_pctg'] = df_copy['avg_monthly_ct_pctg'].mean()
    
    #create individual multiplier for each row
    df_copy['indiv_multiplier'] = (1 - df_copy['avg_monthly_ct_pctg'])/df_copy['avg_monthly_ct_pctg']
    
    #create average multiplier over all rows 
    df_copy['avg_multiplier'] = df_copy['indiv_multiplier'].mean()
    
#if any multipliers are 0 -> manually change to 1 (we'll still get 0 -> 0/1)
    
    return df_copy,df_copy['indiv_multiplier'].mean()
#index 0 -> gives you first return item, 1 gives you second return item

## Eval Error Function

In [ ]:
def eval_error(df):
    #create sample column 
    
    #give two dataframes -> first is overall, second is dataframe that's output of previous function 
    #calculate out samp based on difference bw df1 and ^that df
    
    df_copy = df
    df_copy['sample'] = 'in samp'
    df_copy['sample'].loc[df['Year'] == 2021] = 'out samp'
    
    #get rid of this
    #run removal function
    df_removed = removal(df_copy)
    df_in_samp = df_removed.loc[df_removed['sample'] == 'in samp']
    df_out_samp = df_removed.loc[df_removed['sample'] == 'out samp']
    
    #get multiplier
    mult = actual_multiplier(df_in_samp)
    
    df_out_samp['projected'] = df_out_samp['all_conversions_ct'] * mult
    df_out_samp['abs_error'] = abs(df_out_samp['projected'] - df_out_samp['all_conversions_vt'])
    df_out_samp['abs_error_pctg'] = df_out_samp['abs_error']/ df_out_samp['all_conversions_vt']
    return df_out_samp.sort_values(by= ['abs_error_pctg'])

In [ ]:
# almost half of them are null
df_social_test = df_social
df_social_error = eval_error(df_social_test)
df_social_error.shape

In [ ]:
df_social_error['abs_error_pctg'].notnull().sum()

In [ ]:
df_social_error

# Calculating Multipliers (Split by Channel, Aggregated by Month)

## Display Multiplier -> 153.30

In [ ]:
#remove junk
df_display_removed = removal(df_display)

In [ ]:
df_display.shape

In [ ]:
df_display_removed.shape

In [ ]:
actual_multiplier(df_display_removed)

In [ ]:
df_multiplier(df_display_removed)

## Native Multiplier 

In [ ]:
#remove junk
df_native_removed = removal(df_native)

In [ ]:
df_native.shape

In [ ]:
df_native_removed.shape

In [ ]:
actual_multiplier(df_native_removed)

In [ ]:
df_multiplier(df_native_removed)

## Display and Native Multiplier

In [ ]:
df_display_native_removed = removal(df_display_native)

In [ ]:
df_display_native.shape

In [ ]:
df_display_native_removed.shape

In [ ]:
actual_multiplier(df_display_native_removed)

In [ ]:
df_multiplier(df_display_native_removed)

## Social Multiplier

In [ ]:
df_social_removed = removal(df_social)

In [ ]:
df_social.shape

In [ ]:
df_social_removed.shape

In [ ]:
actual_multiplier(df_social_removed)

In [ ]:
df_multiplier(df_social_removed)

## Facebook Multiplier

In [ ]:
df_fb_removed = removal(df_fb)

In [ ]:
df_fb.shape

In [ ]:
df_fb_removed.shape

In [ ]:
actual_multiplier(df_fb_removed)

In [ ]:
df_multiplier(df_fb_removed)

## Snap Multiplier

In [ ]:
df_snap_removed = removal(df_snap)

In [ ]:
df_snap.shape

In [ ]:
df_snap_removed.shape

In [ ]:
actual_multiplier(df_snap_removed)

In [ ]:
df_multiplier(df_snap_removed)

## Twitter Multiplier

In [ ]:
df_twitter_removed = removal(df_twitter)

In [ ]:
df_twitter.shape

In [ ]:
df_twitter_removed.shape

In [ ]:
actual_multiplier(df_twitter_removed)

In [ ]:
df_multiplier(df_twitter_removed)

# Data Cleaning Section at Top

get rid of Paid Search, spend 0, combine low-volume channels, sites, etc.
centralize and consolidate -> make adjustments before splitting into channel dfs 

# Functions

make more dynamic
training range 2020, out of sample -> 2021 (could potentially adjust)
run the actual functions and run for each channel (at each granularity) 
and get % error table

# Appendix

In [ ]:
# Attempts to group high vol and low vol (that didn't rly work)


#group low-volume channels, products, and sites, and split out channel-Social
# def new_channel(channel):
#     if (channel == 'Social') | (channel == 'Display') | (channel == 'Paid Search') | (channel == 'Native'):
#         return channel
#     else:
#         return 'Other'
    
# # for x in df_copy['channel']:
# #   print(new_channel(x))
    
# df_copy['new_channel'] = df_copy.apply(lambda x: new_channel(x['channel'],axis=1))
#~ -> not

In [ ]:
# REMOVAL FUNCTION (switched 7.12.21)

#returns df with outliers, low-vol channels, sites, and products removed
# # pretty much don't need -> move elsewhere 

# #impact of removals: 

# #if ALL (outliers, paid search, all 3 low vol) removed, then the df is cut in more than half (832638 -> 374316)
# #Outliers (702,551)
# #Paid Search (657,082)
# #Total Conversions = 0 (613,001)
# #Low-volume channels (772,840)
# #low-volume sites (831,519)
# #low-volume products (754,941)
# #all 3 low-volume (733,921)

# # some of it move to the top and remove once -> Paid Search
# # channels where there is 0 spend -> remove it for each specific dataframe 

# def removal(df):
#     #remove_outliers
#     df_spend = df
#     q1, q3 = np.percentile(df_spend['spend'], [25, 75])
#     iqr = q3 - q1
#     lower_bound = q1 - (iqr * 1.5)
#     upper_bound = q3 + (iqr * 1.5)
#     df_outliers_remov= df[df['spend'] > lower_bound][df['spend'] < upper_bound] 

#     # df_spend['spend'].describe(percentile = )
    
#     #remove Paid Search
#     df_paid_search_remov = df_outliers_remov.loc[df['channel'] != 'Paid Search']
    
# #     #remove low-volume channels
# #     df_high_vol_channel = df_paid_search_remov.loc[df['new_channel'] != 'Low-Volume Channels']
    
# #     #remove low-volume sites
# #     df_high_vol_sites = df_high_vol_channel.loc[df['new_site'] != 'Low-Volume Sites']
    
# #     #remove low-volume products
# #     df_high_vol_products = df_high_vol_sites.loc[df['new_product'] != 'Low-Volume Products']
    
#     #remove any June 2020 for Native -> infinity (spend = 0)
#     df_remov_native_zero = df_high_vol_products.loc[~((df['year_month_cal'] == '2020-06-01') & 
#                                              (df['channel'] == 'Native'))]
    
#     #remove any June 2020 for Twitter -> infinity (spend = 0)
#     df_remov_twitter_zero = df_remov_native_zero.loc[~((df['year_month_cal'] == '2020-06-01') & 
#                                              (df['channel'] == 'Social') &
#                                              (df['site'] == 'Twitter'))]
#     #remove total conversions = 0 
#     df_final_remov = df_remov_twitter_zero.loc[df['total_conversions'] == 0]
    
#     return df_final_remov